In [ ]:
%pip install langchain
%pip install -U langchain-community tavily-python
%pip install -uq langchain-openai tavily-python
%pip install beautifulsoup4
%pip install faiss-cpu
%pip install langchainhub
%pip install -U wandb

In [8]:
import os
# import yaml

# config = yaml.safe_load(open("myconfig.yml"))

# os.environ["OPENAI_API_KEY"] = config["OPEN_AI_API_KEY"]
# os.environ["TAVILY_API_KEY"] = config["TAVILY_API_KEY"]
# os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
# os.environ["LANGCHAIN_HUB_API_KEY"] = config["LANGCHAIN_API_KEY"]
# os.environ["LANGCHAIN_TRACING_V2"] = str(config["LANGCHAIN_TRACING_V2"]).lower()
# os.environ["LANGCHAIN_ENDPOINT"] = config["LANGCHAIN_ENDPOINT"]
# os.environ["LANGCHAIN_HUB_API_URL"] = config["LANGCHAIN_HUB_API_URL"]
# os.environ["LANGCHAIN_WANDB_TRACING"] = str(config["LANGCHAIN_WANDB_TRACING"]).lower()
# os.environ["WANDB_PROJECT"] = config["WANDB_PROJECT"]

# Import and load environment variables
from dotenv import load_dotenv
import getpass
load_dotenv()
#Python library used to display progress bars for loops
from tqdm.auto import tqdm


# Planning

![images/Screenshot 2024-02-21 at 16.17.22.png](<images/Screenshot 2024-02-21 at 16.17.22.png>)

You might have come across various techniques aimed at improving the performance of large language models, such as offering tips or even jokingly threatening them. One popular technique is called "chain of thought," where the model is asked to think step by step, enabling self-correction. This approach has evolved into more advanced versions like "chain of thought with self-consistency" and the generalized "tree of thoughts," where multiple thoughts are created, re-evaluated, and consolidated to provide an output.

### Tree of Thoughts

The [@astropomeai tutorial](https://medium.com/@astropomeai/implementing-the-tree-of-thoughts-in-langchains-chain-f2ebc5864fac) on Tree of Thoughts is used as basis of this exercise but expanded with LLMOps tools.

In [4]:
from langchain.chains import LLMChain
# from langchain.llms import OpenAI
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
# from langchain.chat_models import ChatOpenAI
from langchain_ollama import ChatOllama

In [5]:
from langchain import hub

cot_step1 = hub.pull("rachnogstyle/nlw_jan24_cot_step1")
cot_step2 = hub.pull("rachnogstyle/nlw_jan24_cot_step2")
cot_step3 = hub.pull("rachnogstyle/nlw_jan24_cot_step3")
cot_step4 = hub.pull("rachnogstyle/nlw_jan24_cot_step4")

/Users/Shatten/.pyenv/versions/3.11.2/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/Users/Shatten/.pyenv/versions/3.11.2/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/Users/Shatten/.pyenv/versions/3.11.2/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/Users/Shatten/.pyenv/versions/3.11.2/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [6]:
# model = "gpt-3.5-turbo"
model = OllamaLLM(model="llama3.2-vision")

# chain1 = LLMChain(
#     llm=ChatOpenAI(temperature=0, model=model),
#     prompt=cot_step1,
#     output_key="solutions"
# )

chain1 = LLMChain(
    llm=ChatOllama(temperature=0, model="llama3.2-vision"),
    prompt=cot_step1,
    output_key="solutions",
    # other params ...
)

# chain2 = LLMChain(
#     llm=ChatOpenAI(temperature=0, model=model),
#     prompt=cot_step2,
#     output_key="review"
# )

chain2 = LLMChain( 
    llm = ChatOllama(temperature=0, model="llama3.2-vision"),
    prompt=cot_step2,
    output_key="review",
)

# chain3 = LLMChain(
#     llm=ChatOpenAI(temperature=0, model=model),
#     prompt=cot_step3,
#     output_key="deepen_thought_process"
# )

chain3 = LLMChain( 
    llm = ChatOllama(temperature=0, model="llama3.2-vision"),
    prompt=cot_step3,
    output_key="deepen_thought_process",
)

# chain4 = LLMChain(
#     llm=ChatOpenAI(temperature=0, model=model),
#     prompt=cot_step4,
#     output_key="ranked_solutions"
# )

chain4 = LLMChain( 
    llm = ChatOllama(temperature=0, model="llama3.2-vision"),
    prompt=cot_step4,
    output_key="ranked_solutions",
)

/var/folders/mv/5skqq8p925lbtt_m8d7msnkc0000gn/T/ipykernel_87577/3545555269.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain1 = LLMChain(


In [7]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=["ranked_solutions"],
    verbose=True
)

print(
        overall_chain(
            {
                "input": "human colonization of Mars",
                "perfect_factors": "The distance between Earth and Mars is very large, making regular resupply difficult"
            }
    )
)

/var/folders/mv/5skqq8p925lbtt_m8d7msnkc0000gn/T/ipykernel_87577/3364929189.py:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(




> Entering new SequentialChain chain...

> Finished chain.
{'input': 'human colonization of Mars', 'perfect_factors': 'The distance between Earth and Mars is very large, making regular resupply difficult', 'ranked_solutions': "Your assessment of Solution 1: In-Situ Resource Utilization (ISRU) seems reasonable. Here's a breakdown of your evaluation:\n\n**Strengths:**\n\n* Reduced reliance on Earth-based supplies\n* Increased self-sufficiency\n* Potential for in-situ manufacturing\n\nThese benefits are significant, as they can help reduce the logistical challenges and costs associated with establishing a human settlement on Mars.\n\n**Weaknesses:**\n\n* Technological hurdles\n* Resource availability\n* Environmental concerns\n\nThese challenges are indeed significant, but it's not impossible to overcome them. The strategies you proposed for addressing these challenges (robust R&D, collaboration with experts, monitoring and adaptation) are sound and can help mitigate the risks.\n\n**Pro

### ReAct prompt overview

Let's review [ReAct](https://python.langchain.com/docs/modules/agents/agent_types/react) prompt as it's defined in Langchain.

In [16]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

### Self-ask with search

Let's review [self-ask with search](https://python.langchain.com/docs/modules/agents/agent_types/self_ask_with_search) as it's defined in Langchain.

In [17]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/self-ask-with-search")
prompt.template

'Question: Who lived longer, Muhammad Ali or Alan Turing?\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n\nQuestion: When was the founder of craigslist born?\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n\nQuestion: Who was the maternal grandfather of George Washington?\nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washington?\nIntermediate answer: The mother of George Washington was Mary Ball Washington.\nFollow up: Who was the fathe

Approaches to read next:

**Reflexion** ([Shinn & Labash 2023](https://arxiv.org/abs/2303.11366)) is a framework to equips agents with dynamic memory and self-reflection capabilities to improve reasoning skills.

**Chain of Hindsight** (CoH; [Liu et al. 2023](https://arxiv.org/abs/2302.02676)) encourages the model to improve on its own outputs by explicitly presenting it with a sequence of past outputs, each annotated with feedback.